In [1]:
#establishing environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr, ttest_rel, ttest_ind
from scipy import signal as sig
import matplotlib.patches as mpatches
from scipy.io import loadmat, savemat
from ieeg.auth import Session

In [2]:
#loads all the cat spikes before they are in the GDF
#NOT NECESSARY TO LOAD AFTER GDFS ARE CREATED

catspikes = loadmat('/Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/Cat/Bink_files/catspikes.mat')
numSpikes = catspikes['numSpikes'][0] #spike count per event
spikeStart = catspikes['spikeStart'][0] #array of where the spike starts
spikeStop = catspikes['spikeStop'][0] #spike ends
evStart = catspikes['evStart'][0] #event starts
evStop = catspikes['evStop'][0] #event stops

In [14]:
numSpikes[0][0][0][0]

array([  1,   1,   1,   1,   2,   3,   3,   3,   3,   2,   3,   8,   4,
         4,   4,   9,   4,   9,   7,  11,   8,   1,   7,   6,   8,   4,
         6,   8,  10,   5,   4,   5,   8,   5,  12,  11,   5,   6,  12,
         7,   6,   5,   7,   5,   5,   5,   6,   6,   5,   7,   9,   8,
         7,   7,  10,  10,  10,   9,   8,   9,   8,   9,   9,   7,   7,
        11,   8,   9,  85,   4,   5,   5,   7,   5,   6,   8,   7,   8,
        94,   4,   5,   8,   6,   6,   6,  10,  95,   5,   6,   8,  78,
         2,   4,  83,   4,  90,   3,   1,   6,  69,   1,   8,   6,   6,
         4,   8,   5,  90,   4,  87,   2,   4,   8,  10,   5,   8, 105,
         1,  71,   4,  85,   1,   4,  51,  89,   4, 100,   8,  87,   7,
        96,   9,  82,  87,   9,  86,   9,  69,  10,  86,  11,  89,  83,
        81,  86,  11,  78,  85,  75,  82,  12,  89,  11,  88,  78,  81,
         9,  88,  86,  75,  74,  75,  75,  71,  77,  71,  83,  80,  65,
        80,  85,  86,  71,  84,  80,   1,  77,  79,   1,  86,  7

In [30]:
(numSpikes[0][0][0][0][0])

1

In [58]:
seizure_leaders_C1 = []
seizure_leaders_C2 = []
seizure_leaders_C3 = []
seizure_leaders_C4 = []
seizure_leaders_C5 = []


for i in range(len(numSpikes)):
    for j in range(len(numSpikes[i][0])):
        for k in range(len(numSpikes[i][0][j][0])):
            if numSpikes[i][0][j][0][k] >= 20:
                if i == 0:
                    seizure_leaders_C1.append(np.array([j,k,spikeStart[i][0][j][0][k][0][0], evStart[i][0][j][0][k]]))
                if i == 1:
                    seizure_leaders_C2.append(np.array([j,k,spikeStart[i][0][j][0][k][0][0], evStart[i][0][j][0][k]]))
                if i == 2:
                    seizure_leaders_C3.append(np.array([j,k,spikeStart[i][0][j][0][k][0][0], evStart[i][0][j][0][k]]))
                if i == 3:
                    seizure_leaders_C4.append(np.array([j,k,spikeStart[i][0][j][0][k][0][0], evStart[i][0][j][0][k]]))
                if i == 4:
                    seizure_leaders_C5.append(np.array([j,k,spikeStart[i][0][j][0][k][0][0], evStart[i][0][j][0][k]]))

68
